In [1001]:
import pandas as pd
import numpy as np

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn import metrics

In [1002]:
df = pd.read_csv('data/grades.csv')
df = df.drop(labels = 0, axis = 0) 

In [1003]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
#print(df.dtypes)
possible_grades = ['total_enrolled', 'a+', 'a', 'a-', 'b+', 'b', 'b-', 'c+', 'c', 'p', 'c-', 'd', 'f', 'withdrawal', 'inc/no_grade' ]

for i in possible_grades: #converting values from object to integer 
    df[i] = df[i].astype(int)
    
df['class_name'] = df['subject'].astype(str) + df['course_number'].astype(str)

def writing_intensive(class_num):
    if class_num[-1] == 'W':
        return True
    return False

def honors(class_num):
    if class_num[-1] == 'H':
        return True
    return False

def pure_number(class_num):
    if class_num[-1] == 'W' or class_num[-1] == 'H':
        class_num = class_num[:-1]
    return class_num

df['writing_intensive'] = df['course_number'].apply(writing_intensive)
df['honors'] = df['course_number'].apply(honors)
df['class_num'] = df['course_number'].apply(pure_number)


print(df.shape)

(2668, 26)


In [1004]:
def average_calc(df_in):
    score_1 = (df_in['a'] + df_in['a+']) * 4
    score_2 = (df_in['a-']) * 3.7
    score_3 = df_in['b+'] * 3.3
    score_4 = df_in['b'] * 3
    score_5 = df_in['b-'] * 2.7
    score_6 = df_in['c+'] * 2.3
    score_7 = (df_in['c'] + df_in['p']) * 2
    score_8 = df_in['c-'] * 1.7
    score_9 = df_in['d'] * 1
    score_10 = (df_in['f'] + df_in['withdrawal'] + df_in['inc/no_grade']) * 0

    df_in['actual_average'] = (score_1 + score_2 + score_3 + score_4 + score_5 + score_6 + score_7 + score_8 + score_9 + score_10)/df_in['total_enrolled']
    df_in['reasonably_passing'] = np.where(df_in['actual_average'] > 2.5, 1 , 0)

#     save_as = 'data/improved_grades.csv'
#     df.to_csv(save_as, index=False)

#df.describe()
average_calc(df)

save_as = 'data/improved_grades.csv'
df.to_csv(save_as, index=False)

In [1005]:
df.isnull().sum() 
df.duplicated().sum()
#no duplicate or null values 
df.columns

Index(['term', 'subject', 'course_number', 'course_description',
       'class_section', 'instructor', 'total_enrolled', 'a+', 'a', 'a-', 'b+',
       'b', 'b-', 'c+', 'c', 'p', 'c-', 'd', 'f', 'withdrawal', 'inc/no_grade',
       'average_gpa', 'class_name', 'writing_intensive', 'honors', 'class_num',
       'actual_average', 'reasonably_passing'],
      dtype='object')

In [1006]:
df.reasonably_passing.value_counts(normalize = True)

1    0.703523
0    0.296477
Name: reasonably_passing, dtype: float64

In [1007]:
df = pd.get_dummies(df, columns = ['subject'], drop_first = True)

In [1008]:
selected_features = ['class_num', 'total_enrolled', 'writing_intensive', 'honors', 
       'subject_ANTH', 'subject_ARAB', 'subject_ARTH', 'subject_ARTS',
       'subject_ASTR', 'subject_BALA', 'subject_BIOL', 'subject_BUS',
       'subject_CERT', 'subject_CHEM', 'subject_CHIN',
       'subject_CLAS', 'subject_CMAL', 'subject_CMLIT', 'subject_CO-OP',
       'subject_CSCI', 'subject_DANCE', 'subject_DATA', 'subject_DRAM',
       'subject_EAST', 'subject_ECON', 'subject_ECPCE', 'subject_ECPEL',
       'subject_ECPSE', 'subject_ECPSP', 'subject_EECE', 'subject_ENGL',
       'subject_ENSCI', 'subject_EURO', 'subject_FNES', 'subject_FREN',
       'subject_GEOL', 'subject_GERM',  'subject_GRKMD',
       'subject_HEBRW', 'subject_HIST', 'subject_HNRS', 'subject_HTH',
       'subject_ITAL', 'subject_JAZZ', 'subject_JPNS',
       'subject_KOR', 'subject_LABST', 'subject_LBSCI',
       'subject_LCD', 'subject_LIBR', 'subject_MAM', 'subject_MATH',
       'subject_MEDST', 'subject_MES', 'subject_MUSIC', 'subject_PERM',
       'subject_PHIL', 'subject_PHOTO', 'subject_PHYS', 'subject_PORT',
       'subject_PSCI', 'subject_PSYCH', 'subject_QNS', 'subject_RM',
       'subject_RUSS', 'subject_SEEK', 'subject_SEYS', 'subject_SEYSL',
       'subject_SOC', 'subject_SPAN', 'subject_STPER', 'subject_URBST',
       'subject_WGS']

#'subject_AFST', 'subject_CESL', 'subject_JEWST', 'subject_LATIN', 'subject_GREEK',

In [1009]:
X = df[selected_features]
y = df['actual_average']

In [1010]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

In [1011]:
print('Length of our Training data:', X_train.shape[0], '\nLength of our Testing data:', X_test.shape[0])

Length of our Training data: 2134 
Length of our Testing data: 534


In [1012]:
model = LinearRegression()

In [1013]:
model.fit(X_train, y_train)

LinearRegression()

In [1014]:
y_pred = model.predict(X_test)
y_pred

array([ 2.98172512,  0.91972032,  2.7021992 ,  3.307342  ,  2.8936277 ,
        2.97257961,  3.52250406,  2.94453386,  2.74656676,  2.89403439,
        2.40355661,  1.52368937,  2.78786056,  1.52156341,  3.34801719,
        3.10044443,  3.01851882,  2.32028723,  3.08955781,  2.9743889 ,
        1.88373657,  2.70345571,  1.87958889,  3.405614  ,  3.40003526,
        2.89374137,  3.66350747,  3.19402092,  2.40273843,  3.00823886,
        2.99178659,  3.08624821,  3.43553852,  3.34160325,  2.94695179,
        2.89053918,  3.39764721,  3.80710814,  1.50871062,  3.13826604,
        2.40256154,  2.40459155,  3.40301493,  3.18855031,  1.50993737,
        3.52547153,  2.94859018,  2.40323616,  1.52123692,  2.40275822,
        2.9489442 ,  1.5233061 ,  3.01372758,  3.01265614,  2.70071114,
        2.8347997 ,  1.42745006,  2.97228479,  2.98227053,  3.19649632,
        2.97246089,  2.73294921,  2.94765975,  2.94503357,  1.88003617,
        2.70096092,  3.54767032,  2.40263524,  2.40478941,  3.00

In [1015]:
r_squared = metrics.r2_score(y_test, y_pred)
r_squared

0.3843454526551413

In [1016]:
mae = metrics.mean_absolute_error(y_test, y_pred)
mae

0.5594932840838895

Notes:
-On average, guesses were around ~0.56 grade points off, which is the range of an 'a-' to a 'b+

# Using the model to predict Spring 2021 Grades

In [1017]:
new_df = pd.read_csv('data/grades_spring_2021.csv')

In [1018]:
new_df.rename(columns = {'NBR': 'Course Number', 'W': 'Withdrawal', 'INC/NA': 'INC/NO GRADE', 'TOTAL': 'TOTAL ENROLLED'}, inplace =True)
new_df.columns = new_df.columns.str.replace(' ', '_').str.lower()
new_df['inc/no_grade'] = new_df['inc/no_grade'] + new_df['nc']

print(new_df.columns)

for i in possible_grades: #converting values from object to integer 
    new_df[i] = new_df[i].astype(int)

average_calc(new_df)

Index(['term', 'subject', 'course_number', 'course_name', 'section', 'prof',
       'total_enrolled', 'a+', 'a', 'a-', 'b+', 'b', 'b-', 'c+', 'c', 'p',
       'c-', 'd', 'nc', 'f', 'withdrawal', 'inc/no_grade', 'avg_gpa',
       'unnamed:_23', 'unnamed:_24'],
      dtype='object')


In [1019]:
new_df['writing_intensive'] = new_df['course_number'].apply(writing_intensive)
new_df['honors'] = new_df['course_number'].apply(honors)
new_df['class_num'] = new_df['course_number'].apply(pure_number)

new_df.head()

,term,subject,course_number,course_name,section,prof,total_enrolled,a+,a,a-,...,withdrawal,inc/no_grade,avg_gpa,unnamed:_23,unnamed:_24,actual_average,reasonably_passing,writing_intensive,honors,class_num
0,Spring 2021,ACCT,101,Intro Thry & Prac of Acct I,10,"SOLARSH, E",54,16,14,9,...,5,0,3.802174,NaN,NaN,3.238889,1,False,False,101
1,Spring 2021,ACCT,101,Intro Thry & Prac of Acct I,4,"FEISULLIN, A",52,12,17,6,...,3,3,3.704348,NaN,lol,3.276923,1,False,False,101
2,Spring 2021,ACCT,101,Intro Thry & Prac of Acct I,8,"SOLARSH, E",55,23,13,5,...,0,0,3.620755,NaN,NaN,3.489091,1,False,False,101
3,Spring 2021,ACCT,101,Intro Thry & Prac of Acct I,11,"CHAN, J",55,0,25,10,...,3,0,3.613725,NaN,NaN,3.350909,1,False,False,101
4,Spring 2021,ACCT,101,Intro Thry & Prac of Acct I,5,"FEISULLIN, A",35,11,3,0,...,4,2,3.477778,NaN,NaN,2.682857,1,False,False,101


In [1020]:
new_df = pd.get_dummies(new_df, columns = ['subject'], drop_first = True)
spring_X = new_df[selected_features]

spring_pred = model.predict(spring_X)
new_df['predictions'] = spring_pred

new_df[['actual_average', 'predictions']]

,actual_average,predictions
0,3.238889,2.944395
1,3.276923,2.944435
2,3.489091,2.944376
3,3.350909,2.944376
4,2.682857,2.944771
...,...,...
2481,3.206250,2.342215
2482,3.371429,2.342146
2483,3.026923,2.905661
2484,3.270370,2.821592


In [1021]:
mae = metrics.mean_absolute_error(new_df['actual_average'], spring_pred)
mae

0.5545210549925857

In [1022]:
r_squared = metrics.r2_score(new_df['actual_average'], spring_pred)
r_squared

0.42722239571262366